# Upload Zarr Debug

In [1]:
import xarray_dev as xr
import numpy as np
import os

In [11]:
import logging

In [15]:
import fsspec

In [2]:
from dask.distributed import Client

In [3]:
client = Client()

In [4]:
storage_options = {'account_name':'lfhydrophone', 'account_key':os.environ['AZURE_KEY']}

## Create Large Dataset

In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 62.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42493,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 62.80 GiB
Comm: tcp://127.0.0.1:43173,Total threads: 4
Dashboard: http://127.0.0.1:44853/status,Memory: 7.85 GiB
Nanny: tcp://127.0.0.1:36723,


In [6]:
da1 = xr.DataArray(np.random.rand(100000000), dims=['time'])
da2 = xr.DataArray(np.random.rand(100000000), dims=['time'])
da3 = xr.DataArray(np.random.rand(100000000), dims=['time'])

keys = ['da1','da2','da3']

ds = xr.Dataset(dict(zip(keys, [da1,da2,da3])))

In [12]:
ds = ds.chunk({'time':10000000})

In [14]:
ds

<xarray.Dataset>
Dimensions:  (time: 100000000)
Dimensions without coordinates: time
Data variables:
    da1      (time) float64 dask.array<chunksize=(10000000,), meta=np.ndarray>
    da2      (time) float64 dask.array<chunksize=(10000000,), meta=np.ndarray>
    da3      (time) float64 dask.array<chunksize=(10000000,), meta=np.ndarray>

In [9]:
%%time
ds.to_zarr('test.zarr', mode='a')

1
2 1.430511474609375e-06
3 0.0014536380767822266
4 0.0023107528686523438
5 1.9073486328125e-05
6 0.0001227855682373047
7 1.2159347534179688e-05
8 1.0728836059570312e-05
9 1.3828277587890625e-05
10 0.008086919784545898
11 15.922141313552856
12 0.00047469139099121094
CPU times: user 2.61 s, sys: 2.4 s, total: 5.01 s
Wall time: 15.9 s


In [17]:
fs = fsspec.filesystem('abfs', **storage_options)

In [18]:
fs.delete('abfs://hydrophonedata/test_data.zarr', recursive=True)

In [10]:
%%time
ds.to_zarr('abfs://hydrophonedata/test_data.zarr', storage_options=storage_options, mode='a')

1
2 9.5367431640625e-07
3 1.2776398658752441
4 8.240540504455566
5 9.989738464355469e-05
6 4.326051235198975
7 0.00010776519775390625
8 1.1682510375976562e-05
9 2.1457672119140625e-05
10 13.65297818183899
11 73.51754856109619
12 0.00022602081298828125
CPU times: user 13 s, sys: 3.39 s, total: 16.3 s
Wall time: 1min 41s


In [19]:
%%time
ds.to_zarr('abfs://hydrophonedata/test_data.zarr', storage_options=storage_options, mode='a')

1
2 7.152557373046875e-07
3 2.001901149749756
4 1.9273192882537842
5 9.72747802734375e-05
6 1.8642632961273193
7 9.393692016601562e-05
8 1.52587890625e-05
9 1.5974044799804688e-05
10 10.015682697296143
11 157.26703333854675
12 0.0003457069396972656
CPU times: user 23 s, sys: 4.62 s, total: 27.6 s
Wall time: 2min 53s


In [24]:
ds1 = xr.open_dataset('abfs://hydrophonedata/test_data.zarr', engine='zarr', 
                      backend_kwargs={'storage_options': storage_options}, chunks='auto')

In [28]:
ds1.isel(time=slice(0, 2*10000000)).compute()

<xarray.Dataset>
Dimensions:  (time: 20000000)
Dimensions without coordinates: time
Data variables:
    da1      (time) float64 0.4712 0.09025 0.7857 ... 0.3944 0.03352 0.1235
    da2      (time) float64 0.8185 0.4767 0.3437 0.4212 ... 0.4638 0.3546 0.3858
    da3      (time) float64 0.2151 0.2822 0.9286 0.4673 ... 0.918 0.3767 0.7782